In [24]:
import mysql.connector  # Import MySQL connector
from mysql.connector import Error  # Import Error class for exception handling

def connect_db():
    try:
        connection = mysql.connector.connect(
            host='localhost',  #  host of the databse 
            user='app_user',  # user of the database 
            password='root',  # Database password
            database='UNT_Computer_Science',  #name of the database 
            port=3307  # Database port
        )
        if connection.is_connected():  # Checking the statust of connection  if connected or not 
            print("Connected to the database")  # sucess message of established connection 
        return connection  # Returning of the  connection object
    except Error as e:  # Catching of the  connection error
        print(f"Error: {e}")  # Printingg of error message
        return None  # Returning  None if connection failed

if __name__ == "__main__":  # Main program entry point
    conn = connect_db()  # Connect to database
    if conn is not None:  # Check if connection is established
        conn.close()  # Close the connection

Connected to the database


In [38]:
import mysql.connector 
from mysql.connector import Error

def connect_db(database):
    """Connect to the specified database."""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='app_user',       
            password='root',       
            database=database,
            port=3307             
        )
        if connection.is_connected():
            print("Connected to the database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None
    # Creation table clasrroom 

def create_classroom_table(cursor):
    """Create classroom table in the database."""
    create_classroom_table = """
    CREATE TABLE IF NOT EXISTS classroom (
        building        VARCHAR(15),
        room_number     VARCHAR(7),
        capacity        INT,
        PRIMARY KEY (building, room_number)
    );
    """
    cursor.execute(create_classroom_table)
    print("Classroom table created successfully.")
 # creation of table departmnt 
def create_department_table(cursor):
    """Create department table in the database."""
    create_department_table = """
    CREATE TABLE IF NOT EXISTS department (
        dept_name       VARCHAR(20), 
        building        VARCHAR(15), 
        budget          DECIMAL(12,2) CHECK (budget > 0),
        PRIMARY KEY (dept_name)
    );
    """
    cursor.execute(create_department_table)
    print("Department table created successfully.")

def create_other_tables(cursor):
    """Create other related tables in the database."""
    create_course_table = """
    CREATE TABLE IF NOT EXISTS course (
        course_id       VARCHAR(8), 
        title           VARCHAR(50), 
        dept_name       VARCHAR(20),
        credits         INT CHECK (credits > 0),
        PRIMARY KEY (course_id),
        FOREIGN KEY (dept_name) REFERENCES department (dept_name)
            ON DELETE SET NULL
    );
    """
    create_instructor_table = """
    CREATE TABLE IF NOT EXISTS instructor (
        ID              VARCHAR(5), 
        name            VARCHAR(20) NOT NULL, 
        dept_name       VARCHAR(20), 
        salary          DECIMAL(8,2) CHECK (salary > 29000),
        PRIMARY KEY (ID),
        FOREIGN KEY (dept_name) REFERENCES department (dept_name)
            ON DELETE SET NULL
    );
    """
    create_section_table = """
    CREATE TABLE IF NOT EXISTS section (
        course_id       VARCHAR(8), 
        sec_id          VARCHAR(8),
        semester        ENUM('Fall', 'Winter', 'Spring', 'Summer'), 
        year            INT CHECK (year > 1701 AND year < 2100), 
        building        VARCHAR(15),
        room_number     VARCHAR(7),
        time_slot_id    VARCHAR(4),
        PRIMARY KEY (course_id, sec_id, semester, year),
        FOREIGN KEY (course_id) REFERENCES course (course_id)
            ON DELETE CASCADE,
        FOREIGN KEY (building, room_number) REFERENCES classroom (building, room_number)
            ON DELETE SET NULL
    );
    """
    create_teaches_table = """
    CREATE TABLE IF NOT EXISTS teaches (
        ID              VARCHAR(5), 
        course_id       VARCHAR(8),
        sec_id          VARCHAR(8), 
        semester        ENUM('Fall', 'Winter', 'Spring', 'Summer'),
        year            INT,
        PRIMARY KEY (ID, course_id, sec_id, semester, year),
        FOREIGN KEY (course_id, sec_id, semester, year) REFERENCES section (course_id, sec_id, semester, year)
            ON DELETE CASCADE,
        FOREIGN KEY (ID) REFERENCES instructor (ID)
            ON DELETE CASCADE
    );
    """
    create_student_table = """
    CREATE TABLE IF NOT EXISTS student (
        ID              VARCHAR(5), 
        name            VARCHAR(20) NOT NULL, 
        dept_name       VARCHAR(20), 
        tot_cred        INT CHECK (tot_cred >= 0),
        PRIMARY KEY (ID),
        FOREIGN KEY (dept_name) REFERENCES department (dept_name)
            ON DELETE SET NULL
    );
    """
    create_takes_table = """
    CREATE TABLE IF NOT EXISTS takes (
        ID              VARCHAR(5), 
        course_id       VARCHAR(8),
        sec_id          VARCHAR(8), 
        semester        ENUM('Fall', 'Winter', 'Spring', 'Summer'),
        year            INT,
        grade           VARCHAR(2),
        PRIMARY KEY (ID, course_id, sec_id, semester, year),
        FOREIGN KEY (course_id, sec_id, semester, year) REFERENCES section (course_id, sec_id, semester, year)
            ON DELETE CASCADE,
        FOREIGN KEY (ID) REFERENCES student (ID)
            ON DELETE CASCADE
    );
    """
    create_advisor_table = """
    CREATE TABLE IF NOT EXISTS advisor (
        s_ID            VARCHAR(5),
        i_ID            VARCHAR(5),
        PRIMARY KEY (s_ID),
        FOREIGN KEY (i_ID) REFERENCES instructor (ID)
            ON DELETE SET NULL,
        FOREIGN KEY (s_ID) REFERENCES student (ID)
            ON DELETE CASCADE
    );
    """
    create_time_slot_table = """
    CREATE TABLE IF NOT EXISTS time_slot (
        time_slot_id    VARCHAR(4),
        day             VARCHAR(1),
        start_hr        INT CHECK (start_hr >= 0 AND start_hr < 24),
        start_min       INT CHECK (start_min >= 0 AND start_min < 60),
        end_hr          INT CHECK (end_hr >= 0 AND end_hr < 24),
        end_min         INT CHECK (end_min >= 0 AND end_min < 60),
        PRIMARY KEY (time_slot_id, day, start_hr, start_min)
    );
    """
    create_prereq_table = """
    CREATE TABLE IF NOT EXISTS prereq (
        course_id       VARCHAR(8), 
        prereq_id       VARCHAR(8),
        PRIMARY KEY (course_id, prereq_id),
        FOREIGN KEY (course_id) REFERENCES course (course_id)
            ON DELETE CASCADE,
        FOREIGN KEY (prereq_id) REFERENCES course (course_id)
    );
    """
    
    # Execute all create table commands for other tables
    for command in [
        create_course_table,
        create_instructor_table,
        create_section_table,
        create_teaches_table,
        create_student_table,
        create_takes_table,
        create_advisor_table,
        create_time_slot_table,
        create_prereq_table
    ]:
        cursor.execute(command)
        print(f"{command.split()[2]} table created successfully.")

if __name__ == "__main__":
    conn = connect_db("UNT_Computer_Science")
    if conn is not None:
        cursor = conn.cursor()
        create_classroom_table(cursor)
        create_department_table(cursor)
        create_other_tables(cursor)
        cursor.close()
        conn.close()


Connected to the database
Classroom table created successfully.
Department table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.
IF table created successfully.


# Loading of data into the tables

In [43]:
import mysql.connector
from mysql.connector import Error

def connect_db():
    """Connect to the database."""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='app_user',       # Use 'root' or the created 'app_user'
            password='root',       # Use the correct password
            database='UNT_Computer_Science',
            port=3307              # Specify the correct port
        )
        if connection.is_connected():
            print("Connected to the database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

def load_data_from_sql(connection, sql_file_path):
    """Load data from the specified SQL file."""
    try:
        cursor = connection.cursor()
        with open(sql_file_path, 'r') as file:
            sql_script = file.read()
        for statement in sql_script.split(';'):
            if statement.strip():  # Execute only non-empty statements
                cursor.execute(statement)
        connection.commit()  # Commit changes
        print("Data loaded successfully.")
    except Error as e:
        print(f"Error: {e}")
    finally:
        cursor.close()

if __name__ == "__main__":
    conn = connect_db()
    if conn is not None:
        load_data_from_sql(conn, 'smallRelationsInsertFile.sql')  # Provide the correct path to your SQL file
        conn.close()


Connected to the database
Data loaded successfully.


In [ ]:
import mysql.connector
from mysql.connector import Error
from tabulate import tabulate

# Database connection function (already provided)
def connect_db():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='app_user',
            password='root',
            database='UNT_Computer_Science',
            port=3307
        )
        if connection.is_connected():
            print("Connected to the database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

##View Courses

def view_courses(connection, course_id=None):
    cursor = connection.cursor()

    if course_id:
        # Query for specific course details, including instructor, schedule, and enrolled students
        print(f"Details for Course ID: {course_id}")

        # Fetch course title and department
        cursor.execute("""
            SELECT c.course_id, c.title, d.dept_name, c.credits 
            FROM course c 
            JOIN department d ON c.dept_name = d.dept_name
            WHERE c.course_id = %s
        """, (course_id,))
        course_details = cursor.fetchone()
        if course_details:
            print("\nCourse Information:")
            print(tabulate([course_details], headers=["Course ID", "Title", "Department", "Credits"], tablefmt="pretty"))

            # Fetch instructors for the course
            cursor.execute("""
                SELECT i.ID, i.name 
                FROM instructor i 
                JOIN teaches t ON i.ID = t.ID 
                WHERE t.course_id = %s
            """, (course_id,))
            instructors = cursor.fetchall()
            print("\nInstructors Teaching This Course:")
            print(tabulate(instructors, headers=["Instructor ID", "Instructor Name"], tablefmt="pretty"))

            # Fetch schedule details
            cursor.execute("""
                SELECT s.building, s.room_number, s.semester, s.year, ts.day, ts.start_hr, ts.start_min, ts.end_hr, ts.end_min 
                FROM section s
                JOIN time_slot ts ON s.time_slot_id = ts.time_slot_id
                WHERE s.course_id = %s
            """, (course_id,))
            schedule = cursor.fetchall()
            print("\nCourse Schedule:")
            print(tabulate(schedule, headers=["Building", "Room", "Semester", "Year", "Day", "Start Hour", "Start Min", "End Hour", "End Min"], tablefmt="pretty"))

            # Fetch enrolled students
            cursor.execute("""
                SELECT st.ID, st.name 
                FROM student st 
                JOIN takes tk ON st.ID = tk.ID 
                WHERE tk.course_id = %s
            """, (course_id,))
            students = cursor.fetchall()
            print("\nEnrolled Students:")
            print(tabulate(students, headers=["Student ID", "Student Name"], tablefmt="pretty"))
        else:
            print(f"No details found for Course ID: {course_id}")

    else:
            # Fetch and display all courses without department details
        cursor.execute("""
        SELECT course_id, title, credits 
        FROM course
        """)
        courses = cursor.fetchall()
        
        print("All Courses:")
        print(tabulate(courses, headers=["Course ID", "Title", "Credits"], tablefmt="pretty"))
        
        cursor.close()
    
# Function to view details of a specific course
def view_course_details(connection, course_id):
    cursor = connection.cursor()
    query = """
    SELECT 
        c.course_id AS "Course ID",
        c.title AS "Course Title",
        i.name AS "Instructor Name",
        s.semester AS "Semester",
        s.year AS "Year",
        ts.day AS "Day",
        CONCAT(LPAD(ts.start_hr, 2, '0'), ':', LPAD(ts.start_min, 2, '0'), ' - ', 
               LPAD(ts.end_hr, 2, '0'), ':', LPAD(ts.end_min, 2, '0')) AS "Time",
        GROUP_CONCAT(st.name SEPARATOR ', ') AS "Enrolled Students"
    FROM 
        course c
    LEFT JOIN 
        section s ON c.course_id = s.course_id
    LEFT JOIN 
        teaches t ON s.course_id = t.course_id AND s.sec_id = t.sec_id AND s.semester = t.semester AND s.year = t.year
    LEFT JOIN 
        instructor i ON t.ID = i.ID
    LEFT JOIN 
        takes ta ON s.course_id = ta.course_id AND s.sec_id = ta.sec_id AND s.semester = ta.semester AND s.year = ta.year
    LEFT JOIN 
        student st ON ta.ID = st.ID
    LEFT JOIN 
        time_slot ts ON s.time_slot_id = ts.time_slot_id
    WHERE 
        c.course_id = %s
    GROUP BY 
        c.course_id, s.semester, s.year, ts.day, ts.start_hr, ts.start_min, ts.end_hr, ts.end_min, i.name
    """
    cursor.execute(query, (course_id,))
    course_details = cursor.fetchall()
    headers = ["Course ID", "Course Title", "Instructor Name", "Semester", "Year", "Day", "Time", "Enrolled Students"]

    if course_details:
        print(tabulate(course_details, headers=headers, tablefmt="grid"))
    else:
        print("Course not found.")
    cursor.close()

# New functions to add data to the database
def add_instructor(connection, instructor_id, name, dept_name, salary):
    cursor = connection.cursor()
    
    # Input validation
    try:
        # Validate instructor ID (expecting it to be numeric)
        if not instructor_id.isdigit() or len(instructor_id) > 5:
            raise ValueError("Instructor ID must be a numeric string up to 5 digits.")
        
        # Validate salary (must be numeric and positive)
        salary = float(salary)
        if salary <= 29000:
            raise ValueError("Salary must be greater than 29,000.")
    
    except ValueError as ve:
        print(f"Validation Error: {ve}")
        return

    # Attempt to add the instructor to the database
    try:
        query = "INSERT INTO instructor (ID, name, dept_name, salary) VALUES (%s, %s, %s, %s)"
        cursor.execute(query, (instructor_id, name, dept_name, salary))
        connection.commit()
        print("Instructor added successfully.")
    except Error as e:
        print(f"Database Error: {e}")
        connection.rollback()
    finally:
        cursor.close()

def add_instructor(connection, instructor_id, name, dept_name, salary):
    cursor = connection.cursor()
    try:
        # Prepare the query to insert a new instructor
        query = "INSERT INTO instructor (ID, name, dept_name, salary) VALUES (%s, %s, %s, %s)"
        
        # Execute the query with provided parameters
        cursor.execute(query, (instructor_id, name, dept_name, salary))
        connection.commit()
        print("Instructor added successfully.")
    except Error as e:
        # Handle specific errors based on the exception message
        if "Data too long" in str(e):
            print("Error: One of the input values is too long. Please check the format and try again.")
        elif "Incorrect integer value" in str(e):
            print("Error: Salary must be a valid number and greater than 0.")
        elif "Cannot add or update a child row" in str(e):
            print("Error: The department name must exist in the department table.")
        elif "invalid input syntax" in str(e):  # Catch specific database errors
            print("Error: Please ensure all input values are in the correct format.")
        else:
            print(f"Error: {e}")  # General error message for other exceptions

        connection.rollback()  # Rollback in case of error
    finally:
        cursor.close()

# Add a new course
def add_course(connection, course_id, title, dept_name, credits):
    cursor = connection.cursor()
    try:
        # Prepare the query to insert a new course
        query = "INSERT INTO course (course_id, title, dept_name, credits) VALUES (%s, %s, %s, %s)"
        
        # Execute the query with provided parameters
        cursor.execute(query, (course_id, title, dept_name, credits))
        connection.commit()
        print("Course added successfully.")
    except Error as e:
        # Handle specific errors based on the exception message
        if "Data too long" in str(e):
            print("Error: One of the input values is too long. Please check the format and try again.")
        elif "Incorrect integer value" in str(e):
            print("Error: The credits value must be an integer and greater than 0.")
        elif "Cannot add or update a child row" in str(e):
            print("Error: The department name must exist in the department table.")
        else:
            print(f"Error: {e}")  # General error message for other exceptions

        connection.rollback()  # Rollback in case of error
    finally:
        cursor.close()

# Function to add a new student
def add_student(connection, student_id, name, dept_name, tot_cred):
    cursor = connection.cursor()
    
    # Input validation
    if not student_id.isdigit():
        print("Error: Student ID must be a numeric value.")
        return
    if not isinstance(name, str) or not name.isalpha():
        print("Error: Student Name must contain only alphabetic characters.")
        return
    if not isinstance(dept_name, str):
        print("Error: Department Name must be a valid string.")
        return
    if not isinstance(tot_cred, int) or tot_cred < 0:
        print("Error: Total Credits must be a non-negative integer.")
        return

    try:
        # Insert the student into the student table
        query = """
        INSERT INTO student (ID, name, dept_name, tot_cred)
        VALUES (%s, %s, %s, %s)
        """
        cursor.execute(query, (student_id, name, dept_name, tot_cred))
        connection.commit()
        print("Student added successfully.")
    except Error as e:
        print(f"Database Error: {e}")
        connection.rollback()
    finally:
        cursor.close()
# Assign an instructor to teach a course
def assign_instructor_to_course(connection, instructor_id, course_id, sec_id, semester, year):
    # Create a cursor to execute SQL commands
    cursor = connection.cursor()
    try:
        # SQL query to insert a record into the 'teaches' table
        query = """
        INSERT INTO teaches (ID, course_id, sec_id, semester, year) 
        VALUES (%s, %s, %s, %s, %s)
        """
        # Execute the query with provided parameters
        cursor.execute(query, (instructor_id, course_id, sec_id, semester, year))
        # Commit the transaction to save changes
        connection.commit()
        print("Instructor assigned to course successfully.")
    except Error as e:
        # Print an error message if insertion fails
        print(f"Error: {e}")
        # Roll back the transaction to avoid partial changes
        connection.rollback()
    finally:
        # Close the cursor to free up resources
        cursor.close()
# Function to view all students with their advisors
def view_students_with_advisors(connection):
    cursor = connection.cursor()
    query = """
    SELECT s.ID, s.name, a.i_ID
    FROM student s
    LEFT JOIN advisor a ON s.ID = a.s_ID
    """
    cursor.execute(query)
    students = cursor.fetchall()
    print("Students with Advisors:")
    print(tabulate(students, headers=["Student ID", "Student Name", "Advisor ID"], tablefmt="pretty"))
    cursor.close()
# Function to calculate average salary by department
def average_salary_by_department(connection):
    cursor = connection.cursor()
    query = """
    SELECT d.dept_name, AVG(i.salary) as average_salary
    FROM department d
    JOIN instructor i ON d.dept_name = i.dept_name
    GROUP BY d.dept_name
    """
    cursor.execute(query)
    averages = cursor.fetchall()
    print("Average Salary by Department:")
    print(tabulate(averages, headers=["Department", "Average Salary"], tablefmt="pretty"))
    cursor.close()
# Function to search for students by course
def find_students_by_course(connection, course_id):
    cursor = connection.cursor()
    query = """
    SELECT s.ID, s.name FROM student s
    JOIN takes t ON s.ID = t.ID
    WHERE t.course_id = %s
    """
    cursor.execute(query, (course_id,))
    students = cursor.fetchall()
    print(f"Students enrolled in course {course_id}:")
    print(tabulate(students, headers=["Student ID", "Student Name"], tablefmt="pretty"))
    cursor.close()

# Function to find instructors by course
def find_instructors_by_course(connection, course_id):
    cursor = connection.cursor()
    query = """
    SELECT i.ID, i.name FROM instructor i
    JOIN teaches t ON i.ID = t.ID
    WHERE t.course_id = %s
    """
    cursor.execute(query, (course_id,))
    instructors = cursor.fetchall()
    print(f"Instructors teaching course {course_id}:")
    print(tabulate(instructors, headers=["Instructor ID", "Instructor Name"], tablefmt="pretty"))
    cursor.close()


# Function to view all students or a specific student
def view_students(connection):
    cursor = connection.cursor()
    
    # Ask for user input
    student_id = input("Enter student ID to view specific student (or press Enter to view all students): ").strip()
    
    if student_id:
        # Query to fetch a specific student's information
        query = """
        SELECT ID, name, dept_name, tot_cred 
        FROM student 
        WHERE ID = %s
        """
        cursor.execute(query, (student_id,))
        student = cursor.fetchone()  # Fetch the specific student's record

        if student:
            print("Student Record:")
            print(tabulate([student], headers=["ID", "Name", "Dept Name", "Total Credits"], tablefmt="pretty"))
        else:
            print("No student found with ID:", student_id)
    else:
        # Query to fetch all students
        query = """
        SELECT ID, name, dept_name, tot_cred 
        FROM student
        """
        cursor.execute(query)
        students = cursor.fetchall()  # Fetch all students
        print("Student Records:")
        print(tabulate(students, headers=["ID", "Name", "Dept Name", "Total Credits"], tablefmt="pretty"))

    cursor.close()
# Function to view course sections with room capacity
def view_course_sections_with_capacity(connection):
    cursor = connection.cursor()
    query = """
    SELECT s.course_id, s.sec_id, s.semester, s.year, c.capacity
    FROM section s
    JOIN classroom c ON s.building = c.building AND s.room_number = c.room_number
    """
    cursor.execute(query)
    sections = cursor.fetchall()
    print("Course Sections with Room Capacity:")
    print(tabulate(sections, headers=["Course ID", "Section ID", "Semester", "Year", "Capacity"], tablefmt="pretty"))
    cursor.close()

# Function to find students by minimum credits
def find_students_by_minimum_credits(connection, min_credits):
    cursor = connection.cursor()
    query = """
    SELECT ID, name FROM student
    WHERE tot_cred >= %s
    """
    cursor.execute(query, (min_credits,))
    students = cursor.fetchall()
    print(f"Students with at least {min_credits} credits:")
    print(tabulate(students, headers=["Student ID", "Student Name"], tablefmt="pretty"))
    cursor.close()

# Function to enroll a student in a course or assign an instructor to a course section

def enroll_or_assign_to_course(connection):
    cursor = connection.cursor()
    print("\nEnroll/Assign a Student or Instructor to a Course:")
    print("1. Enroll Existing Student")
    print("2. Assign Existing Instructor")
    choice = input("Select an option (1 or 2): ")

    if choice == '1':  # Enroll an existing student
        student_id = input("Enter Existing Student ID: ")
        course_id = input("Enter Course ID: ")
        sec_id = input("Enter Section ID: ")
        semester = input("Enter Semester (e.g., 'Fall'): ")
        year = int(input("Enter Year (e.g., 2024): "))

        # Check if the student exists
        cursor.execute("SELECT * FROM student WHERE ID = %s", (student_id,))
        student_exists = cursor.fetchone()
        if not student_exists:
            print(f"Student ID {student_id} does not exist.")
            return

        # Check if the student is already enrolled in the course
        check_query = """
        SELECT * FROM takes WHERE ID = %s AND course_id = %s AND sec_id = %s AND semester = %s AND year = %s
        """
        cursor.execute(check_query, (student_id, course_id, sec_id, semester, year))
        existing_enrollment = cursor.fetchone()

        if existing_enrollment:
            print(f"Student {student_id} is already enrolled in course {course_id} section {sec_id} for {semester} {year}.")
        else:
            # Insert into takes table for student enrollment
            insert_query = """
            INSERT INTO takes (ID, course_id, sec_id, semester, year)
            VALUES (%s, %s, %s, %s, %s)
            """
            try:
                cursor.execute(insert_query, (student_id, course_id, sec_id, semester, year))
                connection.commit()
                print(f"Student {student_id} successfully added to course {course_id} section {sec_id} for {semester} {year}.")
            except Exception as e:
                print("Error adding student:", e)
                connection.rollback()

    elif choice == '2':  # Assign an existing instructor
        instructor_id = input("Enter Existing Instructor ID: ")
        course_id = input("Enter Course ID: ")
        sec_id = input("Enter Section ID: ")
        semester = input("Enter Semester (e.g., 'Fall'): ")
        year = int(input("Enter Year (e.g., 2024): "))

        # Check if the instructor exists
        cursor.execute("SELECT * FROM instructor WHERE ID = %s", (instructor_id,))
        instructor_exists = cursor.fetchone()
        if not instructor_exists:
            print(f"Instructor ID {instructor_id} does not exist.")
            return

        # Check if the instructor is already assigned to the course
        check_query = """
        SELECT * FROM teaches WHERE ID = %s AND course_id = %s AND sec_id = %s AND semester = %s AND year = %s
        """
        cursor.execute(check_query, (instructor_id, course_id, sec_id, semester, year))
        existing_assignment = cursor.fetchone()

        if existing_assignment:
            print(f"Instructor {instructor_id} is already assigned to course {course_id} section {sec_id} for {semester} {year}.")
        else:
            # Insert into teaches table for instructor assignment
            insert_query = """
            INSERT INTO teaches (ID, course_id, sec_id, semester, year)
            VALUES (%s, %s, %s, %s, %s)
            """
            try:
                cursor.execute(insert_query, (instructor_id, course_id, sec_id, semester, year))
                connection.commit()
                print(f"Instructor {instructor_id} successfully assigned to course {course_id} section {sec_id} for {semester} {year}.")
            except Exception as e:
                print("Error assigning instructor:", e)
                connection.rollback()
    else:
        print("Invalid choice. Please select 1 or 2.")

    cursor.close()

# Main menu  for adding data
def main_menu():
    while True:
        print("\nMain Menu:")
        print("1. View Courses")
        print("2. Add Student/Instructor/Course") 
        print("3. View Students") 
        print("4. Search") 
        print("5. Enroll/Assign a Student or Instructor to Course")  
        print("6. Exit")
        choice = input("Select an option: ")

        conn = connect_db()
        if conn is None:
            continue
        if choice == '1':
            # Prompt to view all courses or a specific course
            course_id = input("Enter a Course ID to view specific details, or press Enter to view all courses: ")
            if course_id.strip():  # View a specific course with details
                view_courses(conn, course_id=course_id)
            else:  # View all courses
                view_courses(conn) 
        elif choice == '2':
            add_data_menu(conn)  # New submenu for adding data
        elif choice == '3':
            view_students(conn)  # Only view students
        elif choice == '4':
            search_menu(conn)
        elif choice == '5':
            enroll_or_assign_to_course(conn) 
        elif choice == '6':
            break
        else:
            print("Invalid option. Please try again.")
        
        conn.close()

# Submenu for adding data
# Submenu for adding data
def add_data_menu(connection):
    while True:
        print("\nAdd Data Menu:")
        print("1. Add New Student")
        print("2. Add New Instructor")
        print("3. Add New Course")
        print("4. Assign Instructor to Course")
        print("5. Back to Main Menu")
        
        choice = input("Select an option: ")

        if choice == '1':
            student_id = input("Enter Student ID (numeric only): ")
            if not student_id.isdigit():
                print("Error: Student ID must be numeric.")
                continue

            name = input("Enter Student Name (letters only): ")
            if not name.replace(" ", "").isalpha():
                print("Error: Name must contain only letters.")
                continue

            dept_name = input("Enter Department Name: ")
            try:
                tot_cred = int(input("Enter Total Credits: "))
                if tot_cred < 0:
                    raise ValueError("Total Credits cannot be negative.")
            except ValueError as ve:
                print(f"Error: {ve}")
                continue

            add_student(connection, student_id, name, dept_name, tot_cred)

        elif choice == '2':
            instructor_id = input("Enter Instructor ID (numeric only): ")
            if not instructor_id.isdigit():
                print("Error: Instructor ID must be numeric.")
                continue

            name = input("Enter Instructor Name (letters only): ")
            if not name.replace(" ", "").isalpha():
                print("Error: Name must contain only letters.")
                continue

            dept_name = input("Enter Department Name: ")
            try:
                salary = float(input("Enter Salary: "))
                if salary < 29000:
                    raise ValueError("Salary must be greater than 29,000.")
            except ValueError as ve:
                print(f"Error: {ve}")
                continue

            add_instructor(connection, instructor_id, name, dept_name, salary)

        elif choice == '3':
            course_id = input("Enter Course ID (alphanumeric): ")
            if not course_id.isalnum():
                print("Error: Course ID must be alphanumeric.")
                continue

            title = input("Enter Course Title: ")
            dept_name = input("Enter Department Name: ")
            try:
                credits = int(input("Enter Credits: "))
                if credits < 0:
                    raise ValueError("Credits cannot be negative.")
            except ValueError as ve:
                print(f"Error: {ve}")
                continue

            add_course(connection, course_id, title, dept_name, credits)

        elif choice == '4':
            instructor_id = input("Enter Instructor ID (numeric only): ")
            if not instructor_id.isdigit():
                print("Error: Instructor ID must be numeric.")
                continue

            course_id = input("Enter Course ID (alphanumeric): ")
            if not course_id.isalnum():
                print("Error: Course ID must be alphanumeric.")
                continue

            sec_id = input("Enter Section ID (alphanumeric): ")
            if not sec_id.isalnum():
                print("Error: Section ID must be alphanumeric.")
                continue

            semester = input("Enter Semester (e.g., Fall, Spring): ")
            try:
                year = int(input("Enter Year (e.g., 2023): "))
                if year < 1900 or year > 2100:
                    raise ValueError("Year must be between 1900 and 2100.")
            except ValueError as ve:
                print(f"Error: {ve}")
                continue

            assign_instructor_to_course(connection, instructor_id, course_id, sec_id, semester, year)

        elif choice == '5':
            break
        else:
            print("Invalid option. Please try again.")

# Function to search within the application
def search_menu(connection):
    while True:
        print("\nSearch Menu:")
        print("1. View Courses with Department Details")
        print("2. View Students with Advisors")
        print("3. Average Salary by Department")
        print("4. Find Students by Course")
        print("5. Find Instructors by Course")
        print("6. View Course Sections with Capacity")
        print("7. Find Students by Minimum Credits")
        print("8. View Course Details")  
        print("9. Back to Main Menu")
        
        choice = input("Select an option: ")

        if choice == '1':
            view_courses(connection)
        elif choice == '2':
            view_students_with_advisors(connection)
        elif choice == '3':
            average_salary_by_department(connection)
        elif choice == '4':
            course_id = input("Enter Course ID: ")
            find_students_by_course(connection, course_id)
        elif choice == '5':
            course_id = input("Enter Course ID: ")
            find_instructors_by_course(connection, course_id)
        elif choice == '6':
            view_course_sections_with_capacity(connection)
        elif choice == '7':
            min_credits = int(input("Enter minimum credits: "))
            find_students_by_minimum_credits(connection, min_credits)  # Find students by minimum credits

        elif choice == '8':
            course_id = input("Enter Course ID: ")
            view_course_details(connection, course_id)
        elif choice == '9':
            break
        else:
            print("Invalid option. Please try again.")

# Run the main menu
main_menu()



Main Menu:
1. View Courses
2. Add Student/Instructor/Course
3. View Students
4. Search
5. Enroll/Assign a Student or Instructor to Course
6. Exit
